In [1]:
# Sparse matrix로 MovieLens 20m 분석

import numpy as np
import pandas as pd

r_cols = ["user_id", "movie_id", "rating", "timestamp"]
# 20M data 읽어오기
ratings = pd.read_csv(
    "./ml-100k/ratings-20m.csv", names=r_cols, sep=",", encoding="latin-1"
)
ratings = ratings[["user_id", "movie_id", "rating"]].astype(int)  # timestamp 제거

# train test 분리
from sklearn.utils import shuffle

TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

##### Sparse matrix 사용 >>>>>>>>>>>>>>>>>>
from scipy.sparse import csr_matrix

data = np.array(ratings["rating"])
row_indices = np.array(ratings["user_id"])
col_indices = np.array(ratings["movie_id"])
ratings = csr_matrix((data, (row_indices, col_indices)), dtype=int)
# <<<<<<<<<<<<<<< Sparse matrix 사용

In [2]:
# New MF class for training & testing
class NEW_MF:
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = ratings
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    # train set의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = (
            self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        )
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = r - prediction

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

    # Test set을 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):  # test 데이터에 있는 각 데이터에 대해서
            x, y, z = ratings_test.iloc[i]  # user_id, item_id, rating 가져오기
            test_set.append([x, y, z])
            self.R[x, y] = 0  # Setting test set ratings to 0
        self.test_set = test_set
        return test_set  # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error / len(self.test_set))

    # Training 하면서 test set의 정확도를 계산
    def test(self):
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1.0 / self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1.0 / self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i + 1, rmse1, rmse2))
            if self.verbose:
                if (i + 1) % 10 == 0:
                    print(
                        "Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f"
                        % (i + 1, rmse1, rmse2)
                    )
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(user_id, item_id)

In [3]:
# Testing MF RMSE
R_temp = ratings.copy()
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=190, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

KeyboardInterrupt: 